In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {
    'n_estimators' : [10 , 150, 300],
    'max_depth' : [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,81.064784,1.244204,0.587588,0.039489,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.977558,0.974843,0.967655,0.972147,0.973773,0.003574,1
8,84.250697,2.879893,0.705384,0.064064,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976661,0.979354,0.973944,0.968553,0.969452,0.973593,0.004130,2
10,46.190187,1.913563,0.463461,0.055416,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.975763,0.975741,0.968553,0.969452,0.973414,0.003672,3
7,42.542072,1.726378,0.512251,0.068015,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.973070,0.974843,0.967655,0.972147,0.973234,0.003527,4
5,69.604147,0.896098,0.624010,0.071767,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.977558,0.973968,0.971249,0.968553,0.970350,0.972336,0.003142,5


In [4]:
rf = RandomForestClassifier()
param = {
    'n_estimators' : [10 , 150, 300],
    'max_depth' : [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,48.338934,1.858759,0.538346,0.094059,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.971275,0.973944,0.969452,0.971249,0.972875,0.003137,1
8,89.100563,2.716271,0.792385,0.161124,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.975763,0.973046,0.966757,0.970350,0.972874,0.004082,2
10,47.954491,2.694543,0.488025,0.042596,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.974865,0.972147,0.968553,0.970350,0.972695,0.003203,3
3,5.693890,0.634998,0.249426,0.044551,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.977558,0.969479,0.977538,0.966757,0.967655,0.971797,0.004777,4
11,79.981768,2.272873,0.559920,0.045819,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975763,0.972172,0.973944,0.967655,0.968553,0.971618,0.003098,5
